In [8]:
import numpy as np
import xarray as xr
import cfgrib
import glob as g
import matplotlib.pyplot as plt
from functools import partial
import time

import my_funcs


import gc

## Main program

In [16]:
#def main():

import gc
### hourly sf temperate dir
paths_Th = "/pool/data/ERA5/E5/sf/an/1H/167/*2019*.grb"
### daily sf data dirs
paths_dew = "/pool/data/ERA5/E5/sf/an/1D/168/*2019*.grb"
paths_sf_pr = "/pool/data/ERA5/E5/sf/an/1D/134/*2019*.grb"
paths_msl_pr = "/pool/data/ERA5/E5/sf/an/1D/151/*2019*.grb"
paths_tot_prec = "/pool/data/ERA5/E5/sf/fc/1D/228/*2019*.grb"
### daily pl horizontal wind speeds dirs
paths_u = "/pool/data/ERA5/E5/pl/an/1D/131/*2019*.grb"
paths_v = "/pool/data/ERA5/E5/pl/an/1D/132/*2019*.grb"

for lon,lat in [(10,12)]:
        ### opening datafiles
        T_max, T_min, T_avg = open_h_data(paths_Th, lon, lat)
        #gc.collect()

        T_dew, Pr_sf, P_sl, Prec  = open_d_data(paths_dew, paths_sf_pr, paths_msl_pr, paths_tot_prec, lon, lat)
        #gc.collect()

        U_sq = open_pl_data(paths_u, paths_v, lon, lat)
        #gc.collect()
        
        ### combine datasets to dataframe
        df_Feat = combine_ds(T_max, T_min, T_dew, T_avg, Prec, Pr_sf, P_sl, U_sq)
        ### creating HW flag
        T_max_np = df_Feat["t2m_max"]

        T_thr = np.quantile(T_max_np, q=0.95)
        T_top = T_max_np > T_thr
        hw_count = count_cons(T_top)
        beta = hw_count>=3  ### define HW for at least 3 cons days

        ### creating the (constant) Threshold temperature
        T_thr_rep = np.repeat(T_thr,len(beta))  ### passing the threshold temperature

        ### creating DOY parameter
        DOY = df_Feat.index.dayofyear

        ### adding beta, DOY and T_thr_rep to dataframe
        df_Feat.insert(len(df_Feat.columns),'DOY',DOY)
        df_Feat.insert(0,'beta',beta)
        df_Feat.insert(8,'T_thr',T_thr_rep)

        ###printing to csv
        print(df_Feat)

             beta     t2m_max     t2m_min         d2m     t2m_avg        tp   
time                                                                          
2019-01-01  False  260.234863  253.425583  252.592957  256.089264  0.000069  \
2019-01-02  False  262.829865  251.618881  253.061081  256.489594  0.000008   
2019-01-03  False  260.000671  251.888855  248.557846  255.101120  0.000000   
2019-01-04  False  263.879700  254.751343  250.426849  258.297028  0.000168   
2019-01-05  False  263.003235  254.986679  251.265289  257.926300  0.000076   
...           ...         ...         ...         ...         ...       ...   
2019-12-27  False  266.024597  259.336853  257.133484  262.036041  0.000000   
2019-12-28  False  263.704041  248.883087  253.938187  256.899445  0.000809   
2019-12-29  False  253.462494  249.042984  243.745850  250.901810  0.000000   
2019-12-30  False  259.993317  249.886337  243.417313  254.951736  0.000000   
2019-12-31  False  268.651764  257.359802  249.17355

# Executing main stepwise

## Loading DKRZ available data

In [2]:
### Util funcs


## great circle distance (central angle)
def _circ_dist(lon1, lat1, lon2, lat2):
    delta_lat = lat2-lat1
    return(np.arccos(np.sin(lon1)*np.sin(lon2) + np.cos(lon1)*np.cos(lon2)*np.cos(delta_lat)))

## slower since _circ_dist is calculated for each file
def _preprocess(x, lon, lat):
    _idx = _circ_dist(x.longitude, x.latitude, lon, lat).argmin()     
    return x.isel( values = _idx,drop=True )

## prob. faster to first load the idx
def _preprocess2(x, idx):
    return x.isel( values = idx,drop=True )

def _extract_idx(path, lon, lat):
    ds_idx = xr.open_dataset(path, indexpath='')
    return _circ_dist(ds_idx.longitude, ds_idx.latitude, lon, lat).argmin() 

def count_cons(tru_arr):
    N = len(tru_arr)
    _counter = 0
    _out = np.zeros(N, dtype = "int")
    for i in range(N):
        if tru_arr[i]:
            _counter += 1
        else:
            _out[i-_counter:i] = _counter 
            _counter = 0
    #if not _counter == 0:
    _out[N-_counter:] = _counter
    return _out


## Open hourly data:
this is hourly  t2m (temperature at 2m above surface) of ERA5 (index number 167)

In [ ]:
paths_Th = "/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2019-01*.grb"

In [3]:
def open_h_data(paths_Th,lon, lat):
    ds = xr.open_mfdataset(paths_Th , preprocess = lambda x: _preprocess(x, lon, lat) , indexpath='');
    T_max=ds.drop(
        ['step', 'surface', 'number', 'valid_time']).resample(
    time="1D").max(dim="time").rename({"t2m": "t2m_max"})
    T_min=ds.drop(
        ['step', 'surface', 'number', 'valid_time']).resample(
        time="1D").min(dim="time").rename({"t2m": "t2m_min"})
    T_avg=ds.drop(
        ['step', 'surface', 'number', 'valid_time']).resample(
        time="1D").mean(dim="time").rename({"t2m": "t2m_avg"})
    return T_max, T_min, T_avg


#ds_Feat = xr.merge([T_max, T_min, T_avg]) 



## Opening daily data

### @ Surface

In [51]:
paths_dew = "/pool/data/ERA5/E5/sf/an/1D/168/E5sf00_1D_2019-01*.grb"
paths_sf_pr = "/pool/data/ERA5/E5/sf/an/1D/134/E5sf00_1D_2019-01_*.grb"
paths_msl_pr = "/pool/data/ERA5/E5/sf/an/1D/151/E5sf00_1D_2019-01_*.grb"
paths_tot_prec = "/pool/data/ERA5/E5/sf/fc/1D/228/E5sf12_1D_2019-01*.grb"

In [4]:
def open_d_data(paths_dew, paths_sf_pr, paths_msl_pr, paths_tot_prec, lon, lat):
    T_dew = xr.open_mfdataset(paths_dew,
                              preprocess = lambda x: _preprocess(x, lon, lat),
                              indexpath='').drop(['step', 'surface', 'valid_time']).resample(time='1D').first();
    Pr_sf = xr.open_mfdataset(paths_sf_pr,
                              preprocess = lambda x: _preprocess(x, lon, lat),
                              indexpath='').drop(['step', 'surface', 'valid_time']).resample(time='1D').first();
    P_sl = xr.open_mfdataset(paths_msl_pr,
                              preprocess = lambda x: _preprocess(x, lon, lat),
                              indexpath='').drop(['step', 'surface', 'valid_time']).resample(time='1D').first();
    Prec = xr.open_mfdataset(paths_tot_prec,
                              preprocess = lambda x: _preprocess(x, lon, lat),
                              indexpath='').drop(['step', 'surface', 'valid_time']).resample(time='1D').first();
    return T_dew, Pr_sf, P_sl, Prec



### @ Pressure levels

In [53]:
paths_u = "/pool/data/ERA5/E5/pl/an/1D/131/*2019-01*.grb"
paths_v = "/pool/data/ERA5/E5/pl/an/1D/132/*2019-01*.grb"

In [5]:
def open_pl_data(paths_u, paths_v, lon, lat):
    u_wind = xr.open_mfdataset(paths_u,
                              preprocess = lambda x: _preprocess(x, lon, lat),
                              indexpath='').drop(['step',  'valid_time']).resample(time='1D').first();
    v_wind = xr.open_mfdataset(paths_u,
                              preprocess = lambda x: _preprocess(x, lon, lat),
                              indexpath='').drop(['step',  'valid_time']).resample(time='1D').first();
    
    bot_lay = u_wind.isobaricInhPa==1000
    U_sq = (v_wind.isel(isobaricInhPa=bot_lay, drop=True)**2+u_wind.isel(isobaricInhPa=bot_lay, drop=True)**2).drop("isobaricInhPa").squeeze()
    return U_sq

## Combining Data
and outputting a pandas.dataframe

In [6]:
def combine_ds(T_max, T_min, T_dew, T_avg, Prec, Pr_sf, P_sl, U_sq):
    ds_Feat = xr.merge([T_max, T_min, T_dew, T_avg, Prec, Pr_sf, P_sl, U_sq])
    return ds_Feat.to_dataframe()

### Creating heatwave flag

In [96]:
#T_thr = ds_Feat.t2m_max.chunk(dict(time=-1)).quantile(0.95).to_numpy() 
#T_max_np = ds_Feat.t2m_max.to_numpy()
T_max_np = df_Feat["t2m_max"]

T_thr = np.quantile(T_max_np, q=0.95)
T_top = T_max_np > T_thr
T_top[-2:-1] = True 
hw_count = count_cons(T_top)
beta = hw_count>=3

### Creating day of the year

In [94]:
DOY = df_Feat.index.dayofyear

## Addind to Dataframe:
after adding ```DOY```, ```beta``` and ```T_thr_rep```
now ```df_Feat``` holds: \
\
df_Feat=\[beta,Tmax,Tmin,Tdew,Tavg,Usq,Prec,Psf,T_thr,Pmsl,DOY \]

In [106]:
#df_Feat.insert(len(df_Feat.columns),'DOY',DOY)
#df_Feat.insert(0,'beta',beta)
#df_Feat.pop("") ## remove colume with this

In [24]:
st = time.time()
path = g.glob(paths)[0]
idx = _extract_idx(path, 10,12)
ds = xr.open_mfdataset(paths , preprocess = partial(_preprocess,idx=idx), indexpath='');
print( time.time() - st)

1.8530371189117432


## Sorting filepaths with glob

In [16]:
startYear=2000
endYear=2000
dirPath =  pathToT2m_1H


l = sorted(g.glob(dirPath+"*_20*.grb"))
start_idx, *_, end_idx = [
     idx for idx, s in enumerate(l) if ( str(startYear) in s or str(endYear) in s )]
#for file in l[start_idx: end_idx+1]:
for file in l[start_idx: end_idx+1][0:1]:
    #print(file)
    ds = xr.open_dataset(file, engine =  "cfgrib")

Can't create file '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2000-01-01_167.grb.923a8.idx'
Traceback (most recent call last):
  File "/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/lib/python3.10/site-packages/cfgrib/messages.py", line 534, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/lib/python3.10/contextlib.py", line 135, in __enter__
    return next(self.gen)
  File "/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/lib/python3.10/site-packages/cfgrib/messages.py", line 500, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2000-01-01_167.grb.923a8.idx'
Can't read index file '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2000-01-01_167.grb.923a8.idx'
Traceback (most recent call last):
  File "/sw/spack-levante/mambaforge-22.9.0

In [6]:
pathToERA5 = "/pool/data/ERA5/E5/"

pathToT2m_1H = "/pool/data/ERA5/E5/sf/an/1H/167/"
pathT_dew_1H = "/pool/data/ERA5/E5/sf/an/1D/168/"

def loadToDataset(dirPath, startYear, endYear ):
    l = sorted(g.glob(dirPath+"*_20*.grb"))
    start_idx, *_, end_idx = [
        idx for idx, s in enumerate(l) if ( str(startYear) in s or str(endYear) in s )]
    for file in l[start_idx: end_idx+1]:
        ds = xr.open_dataset(file,)
    
    
    #xr.open_mfdataset(l[start_idx: end_idx+1], engine="cfgrib")
    
    
#loadToDataset(pathToT2m_1H,2000,2000)
            
#plt.plot([(r"(20[0-1][0-9]|2020)" in a)  for a in sorted(g.glob(pathTo2mT_1H))])



Can't create file '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2000-01-01_167.grb.923a8.idx'
Traceback (most recent call last):
  File "/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/lib/python3.10/site-packages/cfgrib/messages.py", line 534, in from_indexpath_or_filestream
    with compat_create_exclusive(indexpath) as new_index_file:
  File "/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/lib/python3.10/contextlib.py", line 135, in __enter__
    return next(self.gen)
  File "/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/lib/python3.10/site-packages/cfgrib/messages.py", line 500, in compat_create_exclusive
    fd = os.open(path, os.O_WRONLY | os.O_CREAT | os.O_EXCL)
PermissionError: [Errno 13] Permission denied: '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2000-01-01_167.grb.923a8.idx'
Can't read index file '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2000-01-01_167.grb.923a8.idx'
Traceback (most recent call last):
  File "/sw/spack-levante/mambaforge-22.9.0

In [70]:
sorted(g.glob(pathToT2m_1H+"*_20[0-1][0-9]|*.grb"))+sorted(g.glob(pathToT2m_1H+"*_2020*.grb"))

['/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2019-12-22_167.grb',
 '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2019-12-23_167.grb',
 '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2019-12-24_167.grb',
 '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2019-12-25_167.grb',
 '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2019-12-26_167.grb',
 '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2019-12-27_167.grb',
 '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2019-12-28_167.grb',
 '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2019-12-29_167.grb',
 '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2019-12-30_167.grb',
 '/pool/data/ERA5/E5/sf/an/1H/167/E5sf00_1H_2019-12-31_167.grb']

In [30]:
u"(20[0-1][0-9]|2020)"

'(20[0-1][0-9]|2020)'